In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# Programa funcional para os anos de 1929 - 2004
# É recomendado a exploração inicial do HTML do site do IMDB. Em várias décadas/anos específicos existe certa variação em como a informação é armazenada
# Não foi verificado se as informações retiradas estão 100% corretas PRINCIPALMENTE nas décadas de 20, 30, 40 e 50.
# Alguns anos (1930, por exemplo) possuem muita variação na URL do site, formatação e estruturação HTML;
# Além de possuírem 2 premiações no mesmo ano, por esse motivo serão deixados de lado.

# Headers = disfarce para entrar sem ser bloqueado
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
}

# Uma vez encontrado o nome vencedor da categoria, essa função entrara nesse link e buscará dua nacionalidade
def get_person_details(profile_url):
    person_details = {}
    if not profile_url:
        return person_details

    print(f"    Acessando perfil: {profile_url}")
    try:
        person_response = requests.get(profile_url, headers=headers)
        person_response.raise_for_status() # Boa prática para procurar por erros
        person_soup = BeautifulSoup(person_response.text, 'html.parser')

        # Consegui encontrar o local de nascimento de diretores da década 60 em diante usando span + div
        birth_place_label = person_soup.find('span', class_='ipc-metadata-list-item__label', string='Local de Nascimento')

        if birth_place_label:
            content_container = birth_place_label.find_next_sibling('div', class_='ipc-metadata-list-item__content-container')
            if content_container:
                birth_place_link = content_container.find('a', class_='ipc-link--base')
                if birth_place_link:
                    full_birth_place = birth_place_link.text.strip()
                    parts = [p.strip() for p in full_birth_place.split(',')]
                    nationalidade = parts[-1] if parts else None
                    person_details['Nacionalidade'] = nationalidade
                    print(f"      Encontrado Local de Nascimento: {full_birth_place}, Nacionalidade: {nationalidade}")
                else:
                    print("      Link de local de nascimento não encontrado dentro do container.")
            else:
                print("      Container de conteúdo de local de nascimento não encontrado.")
        else:
            # Alguns mencionados continham sua nacionalidade em locais diferentes do restante (principalmente atores ganhadores do milênio)
            birth_place_link_element = person_soup.find('a', class_='ipc-metadata-list-item__list-content-item', href=lambda href: href and 'birth_place' in href)
            if birth_place_link_element:
                full_birth_place = birth_place_link_element.text.strip()
                parts = [p.strip() for p in full_birth_place.split(',')]
                nationalidade = parts[-1] if parts else None
                person_details['Nacionalidade'] = nationalidade
                print(f"      Encontrado Local de Nascimento (alt): {full_birth_place}, Nacionalidade: {nationalidade}")
            else:
                print("      Nenhum elemento de local de nascimento encontrado no perfil.")

        person_details['Etnia'] = None # No site IMDB etnia não faz parte d portifólio, infelizmente. Isso será abordado mais a frente.

    except requests.exceptions.HTTPError as e:
        print(f"    Erro HTTP ao acessar o perfil {profile_url}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"    Erro de requisição ao acessar o perfil {profile_url}: {e}")
    except Exception as e:
        print(f"    Ocorreu um erro inesperado ao processar o perfil {profile_url}: {e}")

    time.sleep(1.5)

    return person_details


def extrair_vencedores_oscar():
    ano_inicio = 1929
    ano_fim = 2025

    base_url = "https://www.imdb.com/pt/event/ev0000003/{ano}/1/?ref_=ev_acd_tl_yr_1"

    resultados_oscar = []

    categorias_interesse = {
        "Melhor Ator": ["BestActorInALeadingRole", "Melhor Ator Principal"],
        "Melhor Atriz": ["BestActressInALeadingRole", "Melhor Atriz Principal"],
        "Melhor Diretor": ["BestDirector", "Melhor Diretor"],
        "Melhor Ator Coadjuvante": ["BestActorInASupportingRole", "Melhor Ator Coadjuvante", "oscar_best_actor_in_a_supporting_role"],
        "Melhor Atriz Coadjuvante": ["BestActressInASupportingRole", "Melhor Atriz Coadjuvante", "oscar_best_actress_in_a_supporting_role"]
    }
    categorias_de_pessoa = list(categorias_interesse.keys())

    for ano in range(ano_inicio, ano_fim + 1):
        url_ano = base_url.format(ano=ano)
        print(f"\nAcessando: {url_ano}")

        dados_ano = {"Ano": ano}
        encontrou_vencedores_no_ano = False

        try:
            response = requests.get(url_ano, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # categorias com IDs específicos (anos mais antigos)
            for category_key, patterns in categorias_interesse.items():
                for pattern in patterns:
                    if pattern.startswith("oscar_"): # Verifica se a categoria específica corresponde ao desejado
                        element_by_id = soup.find(id=pattern)
                        if element_by_id and category_key not in dados_ano:
                            # A maioria dos vencedores estarão em um link (<a>) próximo ou num bloco específico (cerca de 40%)
                            # Para os IDs de Oscar fornecidos, o vencedor geralmente segue em um `a` com certas classes
                            # Tenta encontrar o elemento vencedor dentro do mesmo bloco que o ID ou um irmão/pai próximo

                            ancestor = element_by_id.find_parent('div', class_='event-widgets__award-section')
                            if not ancestor:
                                ancestor = element_by_id.find_parent('section', class_='ipc-page-section')

                            winner_name = None
                            person_profile_url = None

                            if ancestor:
                                winner_signpost_div = ancestor.find('div', class_='ipc-signpost__text', string='Vencedor')
                                if winner_signpost_div:
                                    content_container = winner_signpost_div.find_parent('div', class_='ipc-metadata-list-item__content-container')
                                    if content_container:
                                        person_name_ul = content_container.find('ul', class_=lambda c: c and 'ipc-inline-list' in c.split())
                                        if person_name_ul:
                                            first_li = person_name_ul.find('li', class_='ipc-inline-list__item')
                                            if first_li:
                                                winner_name_element = first_li.find('a')
                                                if winner_name_element:
                                                    winner_name = winner_name_element.text.strip()
                                                    person_profile_url = "https://www.imdb.com" + winner_name_element['href']

                                                    if winner_name:
                                                        dados_ano[category_key] = winner_name
                                                        encontrou_vencedores_no_ano = True
                                                        if person_profile_url:
                                                            person_data = get_person_details(person_profile_url)
                                                            dados_ano[f"{category_key}_Nacionalidade"] = person_data.get('Nacionalidade')
                                                            dados_ano[f"{category_key}_Etnia"] = person_data.get('Etnia')
                                                        else:
                                                            dados_ano[f"{category_key}_Nacionalidade"] = None
                                                            dados_ano[f"{category_key}_Etnia"] = None
                                                        print(f"  Encontrado {category_key} (via ID): {winner_name}")
                                                        # Aqui eu acredito ter abrangido todas as possibilidades para encontrar o URL do vencedor
                                                        # Caso seja encontrado com a primeira tentativa, o 'break' pulará para a próxima etapa.
                                                        break

            # Lógica principal de busca nas sections para os anos mais recentes e categorias sem IDs específicos
            award_sections = soup.find_all('section', class_='ipc-page-section')

            for section in award_sections:
                section_data_testid = section.get('data-testid', '')
                category_title_element = section.find('h3', class_='ipc-title__text--reduced')
                category_name_from_h3 = None
                if category_title_element:
                    category_name_from_h3 = category_title_element.find('span').text if category_title_element.find('span') else category_title_element.text.strip()

                # Nas primeiras edições do óscar existia a categoria 'assistente de diretor'.
                # Nos anos que essa categoria existia, havia um conflito entre o diretor elencado e o assistente.
                # Caso deseje analisar o assistente será necessário uma mudança de lógica. O programa atual NÃO funciona corretamente considerando essa categoria.
                if "assistant director" in section_data_testid.lower() or \
                   "diretor assistente" in (category_name_from_h3 or "").lower():
                    continue

                matched_category_key = None
                # Aqui verificaremos se as categorias serão preenchidas corretamente após todo ajuste pela divergência de armazenamento de informação
                # entre os anos. Precisa de revisão de lógica se tirar o .lower() ou a estrutura if inteira de diretor assistente a cima.
                for key, patterns in categorias_interesse.items():
                    lower_patterns = [p.lower() for p in patterns]
                    # Verifica se o data-testid, h3, ou o ID da seção bate com os padrões propostos na função extrair_vencedor_oscar
                    section_id = section.get('id', '')
                    if (any(p.lower() in section_data_testid.lower() for p in lower_patterns) or
                        (category_name_from_h3 and any(p.lower() in (category_name_from_h3.lower() or "") for p in lower_patterns)) or
                        (section_id and any(p.lower() == section_id.lower() for p in lower_patterns))):
                        matched_category_key = key
                        break

                if matched_category_key:
                    # Se já encontramos um vencedor para esta chave de categoria neste ano, pular para evitar duplicação
                    if matched_category_key in dados_ano:
                        continue

                    winner_signpost_div = section.find('div', class_='ipc-signpost__text', string='Vencedor')
                    # extração do nome do vencedor e o perfil para ser acessado para procurar sua nacionalidade)
                    if winner_signpost_div:
                        winner_name = None
                        person_profile_url = None

                        # Lógica de extração de nome e URL de perfil para categorias de pessoa á partir de ancestrais comuns
                        ancestor = winner_signpost_div.find_parent('div', class_='ipc-metadata-list-item__content-container')
                        if ancestor:
                            person_name_ul = ancestor.find('ul', class_=lambda c: c and 'ipc-inline-list' in c.split())
                            if person_name_ul:
                                first_li = person_name_ul.find('li', class_='ipc-inline-list__item')
                                if first_li:
                                    winner_name_element = first_li.find('a') # o <a> que contém o nome e o href da inspeção da categoria
                                                                             # Anteriormente, aqui eu fiquei travado por um tempo pois as inspeções
                                                                             # em anos diferentes diferem bastante. Sugiro inspecionar um exemplo por
                                                                             # década para começar a entender a estrutura do site.
                                    if winner_name_element:
                                        winner_name = winner_name_element.text.strip()
                                        person_profile_url = "https://www.imdb.com" + winner_name_element['href']

                        if winner_name:
                            dados_ano[matched_category_key] = winner_name
                            encontrou_vencedores_no_ano = True

                            # Se for uma categoria de pessoa e tivermos o URL do perfil, coletamos a nacionalidade
                            if person_profile_url: # Mais uma vez, devido á diferença de armazenamento de informação do site, precisamos verificar se
                                                   # a matched_category_key (página dos vencedores de óscar) e person_data (página do vencedor em questão)
                                                   # são compatíveis.
                                person_data = get_person_details(person_profile_url)
                                dados_ano[f"{matched_category_key}_Nacionalidade"] = person_data.get('Nacionalidade')
                                dados_ano[f"{matched_category_key}_Etnia"] = person_data.get('Etnia')
                                print(f"    Nacionalidade: {dados_ano.get(f'{matched_category_key}_Nacionalidade', 'N/A')}")
                            else:
                                # Caso não encontre o URL do perfil (algo inesperado)
                                dados_ano[f"{matched_category_key}_Nacionalidade"] = None
                                dados_ano[f"{matched_category_key}_Etnia"] = None

                            print(f"  Encontrado {matched_category_key}: {winner_name}")
                        else:
                            print(f"  Não encontrado nome do vencedor para {matched_category_key} no ano {ano}.")
                    else:
                        print(f"  Não encontrado 'Vencedor' para {matched_category_key} no ano {ano}.")

            if encontrou_vencedores_no_ano:
                resultados_oscar.append(dados_ano)
            else:
                if response.status_code != 404:
                    print(f"  Nenhum vencedor das categorias de interesse encontrado para o ano {ano}.")

        except requests.exceptions.HTTPError as e:
            print(f"  Erro HTTP ao acessar a URL {url_ano}: {e}")
        except requests.exceptions.RequestException as e:
            print(f"  Erro de requisição ao acessar a URL {url_ano}: {e}")
        except Exception as e:
            print(f"  Ocorreu um erro inesperado para o ano {ano}: {e}")

        time.sleep(2.5)

    return resultados_oscar

if __name__ == "__main__":
    dados_completos_oscar = extrair_vencedores_oscar()

    if dados_completos_oscar:
        df_oscar = pd.DataFrame(dados_completos_oscar)
        categorias_principais = [
            "Melhor Ator",
            "Melhor Atriz",
            "Melhor Diretor",
            "Melhor Ator Coadjuvante",
            "Melhor Atriz Coadjuvante"
        ]
        colunas_finais = ['Ano']
        for cat in categorias_principais:
            colunas_finais.append(cat)
            colunas_finais.append(f"{cat}_Nacionalidade")
            colunas_finais.append(f"{cat}_Etnia")
        # Adicionar colunas que podem não ter sido preenchidas em todos os anos
        # A coluna de etnia será preenchida posteriormente no arquivo de ETL
        for col in colunas_finais:
            if col not in df_oscar.columns:
                df_oscar[col] = None
        df_oscar = df_oscar[colunas_finais]
        print("\n--- Primeiras 25 linhas dos dados coletados ---")
        print(df_oscar.head(25).to_string())
        df_oscar.to_csv("ganhadores_oscar_pessoas_com_nacionalidade.csv", index=False)
        print("\nDados salvos em 'ganhadores_oscar_pessoas_com_nacionalidade.csv'")
        print(f"Total de anos com dados coletados: {len(dados_completos_oscar)}")
    else:
        print("\nNenhum dado do Oscar foi coletado.")